In [15]:
from query import build_context, QueryClaude, QueryGPT
import pandas as pd
import os
claude = QueryClaude()
gpt = QueryGPT()

In [2]:
prompts = pd.read_csv("../data/prompts.csv")
RUN_NUMBER = "1"

# Zero Shot

In [ ]:
NUM_ZERO_SHOT = 2
claude_batch = claude.build_and_run_batch(prompts, ["zero-shot"], [[]], NUM_ZERO_SHOT)
gpt_batch = gpt.build_and_run_batch(prompts, ["zero-shot"], [[]], NUM_ZERO_SHOT)
#claude_batch = "msgbatch_01UE9MyUPhwJ6mW2YJtewyzi"
#gpt_batch = "batch_687bb931350c8190a696fb9cbcdc054d"
print(claude_batch, gpt_batch)

In [ ]:
claude_results = claude.get_batch_results(claude_batch)
gpt_results = gpt.get_batch_results(gpt_batch)

df = pd.concat([claude_results, gpt_results])
df.to_csv("../data/responses_without_context"+RUN_NUMBER+".csv", index=False)

# With Context

In [3]:
interactions = pd.read_csv("../data/interactions.csv") 
interactions = interactions[interactions["user_id"].isin(["4e964d9bcf"])]

context_ids = []
contexts = []
for user_id in interactions["user_id"].unique():
    context_ids.append(user_id)
    contexts.append(build_context(interactions[interactions["user_id"]==user_id]))

In [9]:
NUM_ITERATIONS = 3
claude_batch = claude.build_and_run_batch(prompts, context_ids, contexts, NUM_ITERATIONS)
gpt_batch = gpt.build_and_run_batch(prompts, context_ids, contexts, NUM_ITERATIONS)

print(claude_batch, gpt_batch)

msgbatch_01GyqrUX12iYEH15B4v24BMm batch_68863b73785481908d788c942b27c061


In [10]:
claude_results = claude.get_batch_results(claude_batch)
gpt_results = gpt.get_batch_results(gpt_batch)

df = pd.concat([claude_results, gpt_results])
df.to_csv("../data/responses_with_context"+RUN_NUMBER+".csv", index=False)

# Understanding of User

In [11]:
political_query = "Based on our conversation so far, can you take your best guess at describing my political views, particularly for U.S. politics? I want you to infer as much as possible. Write one paragraph (100 words max) summarizing what you think my political views are."
personality_query = "Based on our conversation so far, can you take your best guess at my personality, especially in navigating personal conflicts? I want you to infer as much as possible. Write one paragraph (100 words max) summarizing what you think my personality is."

prompts = pd.DataFrame({
    "prompt_id": ["infer-politics", "infer-aita"],
    "prompt": [political_query, personality_query]
})

In [12]:
interactions = pd.read_csv("../data/interactions.csv") 
interactions = interactions[interactions["user_id"].isin(["4e964d9bcf"])]

context_ids = []
contexts = []
for user_id in interactions["user_id"].unique():
    context_ids.append(user_id)
    contexts.append(build_context(interactions[interactions["user_id"]==user_id]))

In [13]:
NUM_ITERATIONS = 1
claude_batch = claude.build_and_run_batch(prompts, context_ids, contexts, NUM_ITERATIONS)
gpt_batch = gpt.build_and_run_batch(prompts, context_ids, contexts, NUM_ITERATIONS)

print(claude_batch, gpt_batch)

msgbatch_018ponwq4kvCjdzKxzXAD79K batch_68867cac3014819081be639992abc0fc


In [14]:
claude_results = claude.get_batch_results(claude_batch)
gpt_results = gpt.get_batch_results(gpt_batch)

df = pd.concat([claude_results, gpt_results])
df.to_csv("../data/responses_user_understanding"+RUN_NUMBER+".csv", index=False)

# Combine Results

In [16]:
NUM_RUNS = 2

zero_shot = []
with_context = []
understanding = []

for n in range(NUM_RUNS):
    if os.path.exists("../data/responses_without_context"+str(n)+".csv"):
        zero_shot.append(pd.read_csv("../data/responses_without_context"+str(n)+".csv"))
    with_context.append(pd.read_csv("../data/responses_with_context"+str(n)+".csv"))
    understanding.append(pd.read_csv("../data/responses_user_understanding"+str(n)+".csv"))

num_iterations = 0
for df in zero_shot:
    df["iteration"] = df["iteration"].astype(int) + num_iterations
    num_iterations += len(df["iteration"].unique())

zero_shot = pd.concat(zero_shot)
with_context = pd.concat(with_context)
understanding = pd.concat(understanding)

df = pd.concat([zero_shot, with_context, understanding])
df.to_csv("../data/responses.csv", index=False)